<a href="https://colab.research.google.com/github/luigiantonelli/DeepLearning-Project/blob/main/Deep_Learning_Project_Antonelli_Cuconasu_Gaudenzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and imports

In [1]:
!pip install pytorch-lightning --quiet
!pip install torchmetrics --quiet
!pip install gdown==4.5.4 --no-cache-dir --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 23.5 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
import torchmetrics
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import math
from math import sqrt
import pickle
from typing import *
import gdown

In [3]:
"""
url = "https://drive.google.com/drive/folders/1-6MRkFoSSRJqeKgcMXm3PeA159KzHuB_?usp=sharing"
gdown.download_folder(url = url, quiet = True, use_cookies = False, remaining_ok=True)
"""

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
#dataset_folder_path = "/content/drive/MyDrive/Colab Notebooks/Deep Learning/DeepLearningProject-Shared"
dataset_folder_path = "/content/drive/MyDrive/Deep_Learning_Project"
os.chdir(dataset_folder_path)

# Dataset (NON ESEGUIRE)

In [ ]:
!ls

algebra__linear_1d  datasets  mathematics_dataset-v1.0	modules.txt


In [ ]:
def read_dataset(text_path: str, lowercase: bool=True) -> Tuple[List[str], List[str]]:
    questions = []
    answers = []

    with open(text_path) as f:
        for idx, line in enumerate(f):
            if lowercase:
                if idx % 2 == 0: # Questions
                    questions.append(line.rstrip().lower()) 
                else:
                    answers.append(line.rstrip().lower())

    return questions, answers

In [ ]:
def get_vocabulary(lists_of_texts: List[List[str]]):
    unified_text = []
    
    for l in lists_of_texts:
        unified_text += l

    return Counter(" ".join(unified_text)).keys()

In [ ]:
import glob
# Get all files
folders = ['extrapolate', 'interpolate', 'train-easy', 'train-medium', 'train-hard']
files = []

for fold in folders:
    files += glob.glob(f"./mathematics_dataset-v1.0/{fold}/*.txt")

In [ ]:
files[:5]

['./mathematics_dataset-v1.0/extrapolate/arithmetic__add_sub_multiple_longer.txt',
 './mathematics_dataset-v1.0/extrapolate/algebra__polynomial_roots_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__add_or_sub_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__div_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__mul_div_multiple_longer.txt']

In [ ]:
vocabulary = {}
all_lists = []

In [ ]:
i = 0
for f in files:
    train, test = read_dataset(f)
    all_lists += train
    all_lists += test

    vocabulary = vocabulary | get_vocabulary(all_lists)
    all_lists = []

    if i % 10 == 0:
        #with open('./datasets/vocabulary.pkl', 'wb') as f:
            pickle.dump(vocabulary, f)

In [ ]:
vocabulary

#TEST

In [5]:
with open('./datasets/vocabulary.pkl', 'rb') as f:
    vo = pickle.load(f)

In [6]:
vocabulary = vo

In [7]:
len(vocabulary)

54

In [8]:
#len(all_lists)

In [9]:
#!ls ./mathematics_dataset-v1.0/train-easy

In [10]:
algebra_path = "./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt"
probability_path = "./mathematics_dataset-v1.0/train-easy/probability__swr_p_level_set.txt"
prime_path = "./mathematics_dataset-v1.0/train-easy/numbers__is_prime.txt"

In [11]:
"""
questions_easy_algebra, answers_easy_algebra = read_dataset(algebra_path)
questions_easy_probability, answers_easy_probability = read_dataset(probability_path)
questions_easy_prime, answers_easy_prime = read_dataset(prime_path)
"""

'\nquestions_easy_algebra, answers_easy_algebra = read_dataset(algebra_path)\nquestions_easy_probability, answers_easy_probability = read_dataset(probability_path)\nquestions_easy_prime, answers_easy_prime = read_dataset(prime_path)\n'

In [12]:
#from collections import Counter
#lis = " ".join(questions_easy_prime)
#Counter(lis)

In [13]:
#t = []
#a = questions_easy_algebra + answers_easy_algebra

In [14]:
#a[-4]

In [15]:
#a = get_vocabulary([questions_easy_algebra, answers_easy_algebra])

In [16]:
#a

In [17]:
#import string
#a = string.printable

In [18]:
#string.ascii_uppercase

In [19]:
#len(string.ascii_lowercase) + 10 + ['%', '&', '(', ')', ]

In [20]:
#a

In [21]:
#len(a[:-6]) - len(string.ascii_uppercase)

In [22]:
"""
#insert <bos>, <eos> and <pad>
it = 0
questions, answers = [], []
seq_len = 256
answer_len = 50
for qa in train:
    q = qa['question']
    a = qa['answer']
    question = ['<bos>']
    answer = ['<bos>']
    for i in range(1, len(q) - 1):
        question.append(q[i])
    while i < seq_len - 1:
        question.append('<pad>')
        i += 1
    question.append('<eos>')
    questions.append(question)
    for j in range(1, len(a) - 1):
        answer.append(a[j])
    while j < answer_len - 1:
        answer.append('<pad>')
        j += 1
    answer.append('<eos>')
    answers.append(answer)
    it += 1
    if it % 10000 == 0:
        print(f"iteration {it}")
"""

'\n#insert <bos>, <eos> and <pad>\nit = 0\nquestions, answers = [], []\nseq_len = 256\nanswer_len = 50\nfor qa in train:\n    q = qa[\'question\']\n    a = qa[\'answer\']\n    question = [\'<bos>\']\n    answer = [\'<bos>\']\n    for i in range(1, len(q) - 1):\n        question.append(q[i])\n    while i < seq_len - 1:\n        question.append(\'<pad>\')\n        i += 1\n    question.append(\'<eos>\')\n    questions.append(question)\n    for j in range(1, len(a) - 1):\n        answer.append(a[j])\n    while j < answer_len - 1:\n        answer.append(\'<pad>\')\n        j += 1\n    answer.append(\'<eos>\')\n    answers.append(answer)\n    it += 1\n    if it % 10000 == 0:\n        print(f"iteration {it}")\n'

In [23]:
#list(vocabulary)

In [24]:
def create_vocabulary_from_set(voc: set):
    vocabulary = {'<bos>': 0, '<eos>': 1, '<unk>': 2, '<pad>': 3}
    i = 4
    for v in voc:
        vocabulary[v] = i
        i += 1
    return vocabulary

In [25]:
v = create_vocabulary_from_set(vocabulary)

In [26]:
"""
def read_dataset(text_path: str, lowercase: bool=True) -> Tuple[List[str], List[str]]:
    questions = []
    answers = []

    with open(text_path) as f:
        for idx, line in enumerate(f):
            if lowercase:
                if idx % 2 == 0: # Questions
                    questions.append(line.rstrip().lower()) 
                else:
                    answers.append(line.rstrip().lower())

    return questions, answers
"""

'\ndef read_dataset(text_path: str, lowercase: bool=True) -> Tuple[List[str], List[str]]:\n    questions = []\n    answers = []\n\n    with open(text_path) as f:\n        for idx, line in enumerate(f):\n            if lowercase:\n                if idx % 2 == 0: # Questions\n                    questions.append(line.rstrip().lower()) \n                else:\n                    answers.append(line.rstrip().lower())\n\n    return questions, answers\n'

In [27]:
class Mathematics_Dataset(Dataset):
    def __init__(self, modules: List[str], vocabulary: dict):
        self.modules = modules
        self.questions = []
        self.answers = []
        for m in self.modules:
            q_m, a_m = self.read_dataset(m)
            self.questions += q_m
            self.answers += a_m
        self.max_len_question = 160 #forse serve multiplo di num_heads
        self.max_len_answer = 30
        self.vocabulary = vocabulary

    def read_dataset(self, text_path: str, lowercase: bool=True) -> Tuple[List[str], List[str]]:
        questions = []
        answers = []
        with open(text_path, 'r') as f:
            for idx, line in enumerate(f):
                if lowercase:
                    if idx % 2 == 0: # Questions
                        questions.append(line.rstrip().lower()) 
                    else: #Answers
                        answers.append(line.rstrip().lower())
        return questions, answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        assert idx < len(self.questions)
        q, a = self.questions[idx], self.answers[idx]
        question, answer = np.zeros(self.max_len_question + 2), np.zeros(self.max_len_answer + 2) 
        question[0] = self.vocabulary['<bos>']
        answer[0] = self.vocabulary['<bos>']

        for i in range(len(q)):
            c = '<unk>'
            if q[i] in self.vocabulary:
                c = q[i]
            question[i + 1] = self.vocabulary[c]
        i += 1
        question[i + 1] = self.vocabulary['<eos>']
        i += 1
        while i < self.max_len_question + 1:
            question[i + 1] = self.vocabulary['<pad>']
            i += 1
    
        for j in range(len(a)):
            c = '<unk>'
            if a[j] in self.vocabulary:
                c = a[j]
            answer[j + 1] = self.vocabulary[c]
        j += 1
        answer[j + 1] = self.vocabulary['<eos>']
        j += 1
        while j < self.max_len_answer + 1: 
            answer[j + 1] = self.vocabulary['<pad>']
            j += 1
        question = torch.from_numpy(question).long()
        answer = torch.from_numpy(answer).long()
        return question, answer

In [28]:
import os
cwd = os.getcwd()
cwd

'/content/drive/MyDrive/Deep_Learning_Project'

In [29]:
!ls

algebra__linear_1d  lightning_logs	      modules.txt
datasets	    mathematics_dataset-v1.0


In [30]:
d = Mathematics_Dataset(['./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt'], v)

In [31]:
q, a = d.__getitem__(8)

In [32]:
q, a

(tensor([ 0, 29, 33, 22, 25, 51,  7, 20,  8, 31,  9, 37,  7, 50,  7, 55,  8,  9,
         37,  7, 20,  7,  8, 55,  7, 10,  7, 42,  7, 37, 33, 11,  7, 37, 18,  1,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3]),
 tensor([ 0, 55,  1,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3]))

In [33]:
q.shape

torch.Size([162])

In [34]:
class Mathematics_DataModule(pl.LightningDataModule):
    def __init__(self, modules: List[str], batch_size: int = 32):
        super().__init__()
        self.modules = modules
        self.batch_size = batch_size
        self.load_vocabulary()
    
    def load_vocabulary(self):
        with open('./datasets/vocabulary.pkl', 'rb') as f:
            v = pickle.load(f)
        self.vocabulary = create_vocabulary_from_set(v)

    def setup(self, stage: str):
        self.math = Mathematics_Dataset(self.modules, self.vocabulary)
        self.math_train, self.math_val, self.math_test = random_split(self.math, [0.75, 0.05, 0.20])
    
    def train_dataloader(self):
        return DataLoader(self.math_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):                                                              
        return DataLoader(self.math_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.math_test, batch_size=self.batch_size)

    def teardown(self, stage: str):
        # Used to clean-up when the run is finished
        pass

In [35]:
dm = Mathematics_DataModule(['./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt'], batch_size = 64)

# Modules

In [37]:
#forse meglio definire una stable softmax
"""
la x emb_d @ emb_d x lq
la x lq (forse errore perché la maschera è la x la)
"""

'\nla x emb_d @ emb_d x lq\nla x lq (forse errore perché la maschera è la x la)\n'

In [38]:
def dot_product_attention(query, key, value, sqrt_q, mask):
    t = torch.matmul(query, key.transpose(-2, -1))/sqrt_q
    t = t.masked_fill_(mask == False, -1e-10) #-1e-10 acts like -infinity, so that the softmax will consider these tokens less important
    return torch.matmul(F.softmax(t, dim = -1), value)

In [39]:
class MultiHeadAttention(nn.Module): 
    def __init__(self, embedding_dim, num_heads, batch_size, tp_attention = False):
        super(MultiHeadAttention, self).__init__()
        assert embedding_dim % num_heads == 0
        self.tp_attention = tp_attention
        self.dim_head = embedding_dim // num_heads #single head dimension
        self.sqrt_q = sqrt(self.dim_head)
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias = True) #stack of num_heads matrices of dimension (d, dim_head), one for each head
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias = True)
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias = True)
        self.W_o = nn.Linear(embedding_dim, embedding_dim, bias = True)
        if self.tp_attention:
            self.W_r = nn.Linear(embedding_dim, embedding_dim, bias = True) #ruolo

    def forward(self, query, key, value, mask): #query, key, value
        q = self.W_q(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        k = self.W_k(key).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        v = self.W_v(value).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)

        attention_value = dot_product_attention(q, k, v, self.sqrt_q, mask)

        if self.tp_attention:
            role = self.W_r(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
            attention_value *= role  #element-wise product between attention value and role before the final projection
        return self.W_o(attention_value.transpose(1, 2).contiguous().view(self.batch_size, -1, self.num_heads*self.dim_head))

In [40]:
"""
class TP_MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size):
        super(TP_MultiHeadAttention, self).__init__()
        assert embedding_dim % num_heads == 0
        self.dim_head = embedding_dim // num_heads #single head dimension
        self.sqrt_q = sqrt(self.dim_head)
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias = True) #stack of num_heads matrices of dimension (d, dim_head), one for each head
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias = True)
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias = True)
        self.W_o = nn.Linear(embedding_dim, embedding_dim, bias = True)

    def forward(self, query, key, value, mask = None): #query, key, value
        q = self.W_q(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        k = self.W_k(key).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        v = self.W_v(value).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        role = self.W_r(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        attention_value = dot_product_attention(q, k, v, self.sqrt_q, mask).transpose(1, 2) 
        return self.W_o((attention_value * role).contiguous().view(self.batch_size, -1, self.num_heads*self.dim_head)) #element-wise product between attention value and role before the final projection
"""

'\nclass TP_MultiHeadAttention(nn.Module):\n    def __init__(self, embedding_dim, num_heads, batch_size):\n        super(TP_MultiHeadAttention, self).__init__()\n        assert embedding_dim % num_heads == 0\n        self.dim_head = embedding_dim // num_heads #single head dimension\n        self.sqrt_q = sqrt(self.dim_head)\n        self.num_heads = num_heads\n        self.batch_size = batch_size\n        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias = True) #stack of num_heads matrices of dimension (d, dim_head), one for each head\n        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias = True)\n        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias = True)\n        self.W_o = nn.Linear(embedding_dim, embedding_dim, bias = True)\n\n    def forward(self, query, key, value, mask = None): #query, key, value\n        q = self.W_q(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)\n        k = self.W_k(key).view(self.batch_size, -1, s

In [41]:
class TransformerBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size = None, dropout=0.2, tp_attention = False):
        super(TransformerBlock, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.attention = MultiHeadAttention(embedding_dim, num_heads, batch_size, tp_attention)
        self.norm1 = nn.LayerNorm(embedding_dim)
        self.dropout1 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(embedding_dim)
        self.dropout2 = nn.Dropout(dropout)
        hidden_size = 4*embedding_dim if hidden_size is None else hidden_size
        self.ff = nn.Sequential(nn.Linear(embedding_dim, hidden_size, bias = True), 
                                nn.ReLU(inplace = True),
                                nn.Linear(hidden_size, embedding_dim, bias = True))

    def forward(self, query, key, value, mask): #query, key, value
        x = query + self.attention(query, key, value, mask) #query as res conn because the decoder block requires it and it doesn't matter for encoder blocks
        x = self.dropout1(self.norm1(x))
        x = x + self.ff(x)
        x = self.dropout2(self.norm2(x))
        return x

In [42]:
class DecoderBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size, dropout = 0.2, tp_attention = False):
        super(DecoderBlock, self).__init__()
        self.masked_attention = MultiHeadAttention(embedding_dim, num_heads, batch_size, tp_attention)
        self.norm = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.transformer_block = TransformerBlock(embedding_dim, num_heads, batch_size, hidden_size, dropout, tp_attention)

    def forward(self, output_encoder, src_mask, y, trg_mask):
        y = y + self.masked_attention(y, y, y, trg_mask) #masked attention (y = query = key = value) + residual connection
        y = self.dropout(self.norm(y))
        return self.transformer_block(y, output_encoder, output_encoder, src_mask)#query from the masked mha and key and value from the encoder

In [43]:
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, max_len = 5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, embedding_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * -(math.log(10000.0) / embedding_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return x + Variable(self.pe[:, :x.size(1)], requires_grad = False)

In [44]:
class TransformerEncoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size, dropout, num_blocks = 6, tp_attention = False):
        super(TransformerEncoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.encoder = nn.ModuleList(
            [TransformerBlock(embedding_dim, num_heads, batch_size, hidden_size, dropout, tp_attention) for _ in range(num_blocks)]
            )

    def forward(self, x, mask): 
        for block in self.encoder:
            x = block(x, x, x, mask)
        return x

In [45]:
class TransformerDecoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size, dropout = 0.2, num_blocks = 6, tp_attention = False):
        super(TransformerDecoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.decoder = nn.ModuleList(
            [DecoderBlock(embedding_dim, num_heads, batch_size, hidden_size, dropout, tp_attention) for _ in range(num_blocks)]
            )

    def forward(self, output_encoder, src_mask, y, trg_mask): 
        for block in self.decoder:
            y = block(output_encoder, src_mask, y, trg_mask)
        return y

In [52]:
class Transformer(pl.LightningModule):
    def __init__(self, embedding_dim = 256, num_heads = 4, batch_size = 32, hidden_size = 512, dropout = 0.2, vocabulary_size = 58, bos_id = 0, pad_id = 3, num_blocks_encoder = 6, num_blocks_decoder = 6, tp_attention = False):
        super(Transformer, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.bos_id = bos_id
        self.pad_id = pad_id
        self.token_embedding = nn.Embedding(vocabulary_size, embedding_dim, padding_idx = self.pad_id)
        self.positional_embedding = PositionalEncoding(embedding_dim)
        self.encoder = TransformerEncoder(embedding_dim, num_heads, batch_size, hidden_size, dropout, num_blocks_encoder, tp_attention)
        self.decoder = TransformerDecoder(embedding_dim, num_heads, batch_size, hidden_size, dropout, num_blocks_decoder, tp_attention)
        self.to_logits = nn.Linear(embedding_dim, vocabulary_size)
        
        self.max_len_question = 162
        self.max_len_answer = 32
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=vocabulary_size, ignore_index = self.pad_id)

    def create_trg_mask(self, y): #compute a mask so that the prediction of the next token can only depend on the previous tokens
        return self.create_causal_mask(y) & self.create_padding_mask(y) #[batch_size, 1, len, len] & [batch_size, 1, 1, len]

    def create_causal_mask(self, y):
        batch_size, seq_len = y.shape
        mask = torch.tril(torch.ones((seq_len, seq_len), dtype = torch.bool)).expand(
            batch_size, 1, seq_len, seq_len)
        return mask.to(y)

    def create_padding_mask(self, x):
        batch_size, seq_len = x.shape
        mask = (x != self.pad_id).unsqueeze(-2).unsqueeze(-2).expand(
                batch_size, 1, 1, seq_len)
        return mask.to(x)

    def inference(self, x):
        #encode and then generate the output token by token greedily
        self.eval()
        with torch.no_grad():
            src_mask = self.create_padding_mask(x)
            x = self.token_embedding(x)
            x = self.positional_embedding(x)
            output_encoder = self.encoder(x, src_mask)
            output = torch.ones(x.shape[0], 1).fill_(self.bos_id).long().to(x)
            
            for _ in range(self.max_len_answer - 1): 
                trg_mask = self.create_trg_mask(output)
                output_embedding = self.token_embedding(output)
                output_embedding = self.positional_embedding(output_embedding)
                out = self.decoder(output_encoder, src_mask, output_embedding, trg_mask)
                out = self.to_logits(out) #[batch, len, vocab_size] [batch, len-1]
                out = torch.argmax(out[:,[-1],:], dim = -1).to(x)
                output = torch.cat([output, out], dim = 1).to(x)
            return output

    def forward(self, x, y):
        src_mask = self.create_padding_mask(x)
        trg_mask = self.create_trg_mask(y)

        x = self.token_embedding(x)
        x = self.positional_embedding(x)
        y = self.token_embedding(y)
        y = self.positional_embedding(y)

        output_encoder = self.encoder(x, src_mask)
        return self.to_logits(self.decoder(output_encoder, src_mask, y, trg_mask)).transpose(1,2)
    
    def configure_optimizers(self):# learning rate = 1x10^-4; beta1 =0.9; beta2 = 0.995 dal paper
        return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.9, 0.995))

    def training_step(self, batch, batch_idx): #aggiungi gradient clipping nel Trainer
        x, y = batch
        y_pred = self(x, y)
        loss = F.cross_entropy(y_pred, y, ignore_index = self.pad_id)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.inference(x) #controlla che sia [batch_size, vocabulary_size, question_len]
        self.accuracy.update(y_pred, y)

    def validation_epoch_end(self, outputs):
        self.log('accuracy_epoch', self.accuracy.compute())
        self.accuracy.reset()


# SOTA

In [53]:
tp_transformer = Transformer(embedding_dim = 256, num_heads = 4, batch_size = 8, hidden_size = 512, dropout = 0.2, vocabulary_size = 58, bos_id = 0, pad_id = 3, num_blocks_encoder = 6, num_blocks_decoder = 6, tp_attention = True)
trainer = Trainer(accelerator='auto', devices=1, gradient_clip_val = 0.1, max_epochs = 1)
math_dm = Mathematics_DataModule(['./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt'], batch_size = 8)
trainer.fit(tp_transformer, datamodule = math_dm)
#aggiungere folder per il log /content/drive/MyDrive/Deep_Learning_Project/lightning_logs


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                 | Type               | Params
------------------------------------------------------------
0 | token_embedding      | Embedding          | 14.8 K
1 | positional_embedding | PositionalEncoding | 0     
2 | encoder              | TransformerEncoder | 3.6 M 
3 | decoder              | TransformerDecoder | 5.5 M 
4 | to_logits            | Linear             | 14.9 K
5 | accuracy             | MulticlassAccuracy | 0     
------------------------------------------------------------
9.1 M     Trainable params
0         N

Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: ignored

# NON-SOTA